# Size Model

## Loading input

In [1]:
import pickle, os, math, random
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Load the dataset
df = pickle.load(open('dataset/df_2308', 'rb'))

# Sort the data in ascending order based on the 'opdate' column (most recent first)
df.sort_values(by=['opdate'], ascending=True, inplace=True)

# Extract features (x) and target (y) values from the dataFrame
x = df[['age_cal', 'sex', 'weight', 'height', 'cuffed']].astype(float).values
y = df['airway_tube_size'].astype(float).values
c = df['opid'].values

# Separate the data for cuffed and uncuffed endotracheal tubes
## Cuffed endotrahceal tube
x_cuffed = x[x[:, 4] == 1][:, :3]
y_cuffed = y[x[:, 4] == 1]

# Determine the size of the test and train sets for cuffed data
nsamp = len(y_cuffed)
ntest = int(nsamp * 0.2)
ntrain = nsamp - ntest

# Split the cuffed data into training and test sets
x_cuff_test = x_cuffed[-ntest:, :]
y_cuff_test = y_cuffed[-ntest:]
x_cuff_train = x_cuffed[:ntrain, :]
y_cuff_train = y_cuffed[:ntrain]

# Impute missing values in the cuffed data using the multiple imputation method
imp = IterativeImputer().fit(x_cuff_train)
x_cuff_train = imp.transform(x_cuff_train)
x_cuff_test = imp.transform(x_cuff_test)

## Uncuffed endotracheal tube
x_uncuffed = x[x[:, 4] == 0][:, :3]
y_uncuffed = y[x[:, 4] == 0]

# Determine the size of the test and train sets for uncuffed data
nsamp = len(y_uncuffed)
ntest = int(nsamp * 0.2)
ntrain = nsamp - ntest

# Split the uncuffed data into training and test sets
x_uncuff_test = x_uncuffed[-ntest:, :]
y_uncuff_test = y_uncuffed[-ntest:]
x_uncuff_train = x_uncuffed[:ntrain, :]
y_uncuff_train = y_uncuffed[:ntrain]

# Impute missing values in the uncuffed data using the multiple imputation method
imp = IterativeImputer().fit(x_uncuff_train)
x_uncuff_train = imp.transform(x_uncuff_train)
x_uncuff_test = imp.transform(x_uncuff_test)

# Print the shapes of the cuffed and uncuffed training and test sets
print(f'x_cuff_train: {(x_cuff_train).shape}, x_cuff_test: {x_cuff_test.shape}')
print(f'x_uncuff_train: {(x_uncuff_train).shape}, x_uncuff_test: {x_uncuff_test.shape}')

x_cuff_train: (10712, 3), x_cuff_test: (2678, 3)
x_uncuff_train: (18934, 3), x_uncuff_test: (4733, 3)


## Feature Selection (BorutaShap)

### Cuffed model

In [ ]:
import pandas as pd
import xgboost as xgb
from BorutaShap import BorutaShap

# Define the input variables (features) and target variable (label)
INPUT_VARS = ['age', 'sex', 'weight', 'height']
TARGET_VAR = 'airway_tube_size'

# Create a DataFrame with the training features (X) from the cuffed data
X = pd.DataFrame(x_cuff_train, columns=INPUT_VARS)

# Set a random seed for reproducibility
SEED = 98

# Create an XGBoost Regressor model
xgbr = xgb.XGBRegressor()

# Initialize the BorutaShap feature selector
Feature_Selector = BorutaShap(model=xgbr, 
                              importance_measure='shap', 
                              classification=False, 
                              percentile=100, 
                              pvalue=0.05)

# Fit the feature selector to the data
Feature_Selector.fit(X=X, 
                     y=y_cuff_train, 
                     n_trials=100, 
                     sample=False, 
                     train_or_test='train', 
                     normalize=True, 
                     verbose=True, 
                     random_state=SEED)

# Plot the results of the BorutaShap feature selection
Feature_Selector.plot(X_size=10,
                       which_features='all')

In [2]:
# Exclude the 'sex' feature and concatenate 'age', 'weight', and 'height' features into a new feature set
x_cuff_train = np.concatenate((x_cuff_train[:,0:1], x_cuff_train[:,2:4]),axis=-1)
x_cuff_test = np.concatenate((x_cuff_test[:,0:1], x_cuff_test[:,2:4]),axis=-1)

### Uncuffed model

In [ ]:
import pandas as pd
import xgboost as xgb
from BorutaShap import BorutaShap

# Define the input variables (features) and target variable (label)
INPUT_VARS = ['age', 'sex', 'weight', 'height']
TARGET_VAR = 'airway_tube_size'

# Create a DataFrame with the training features (X) from the uncuffed data
X = pd.DataFrame(x_uncuff_train, columns=INPUT_VARS)

# Set a random seed for reproducibility
SEED = 98

# Create an XGBoost Regressor model
xgbr = xgb.XGBRegressor()

# Initialize the BorutaShap feature selector
Feature_Selector = BorutaShap(model=xgbr, 
                              importance_measure='shap', 
                              classification=False, 
                              percentile=100, 
                              pvalue=0.05)

# Fit the feature selector to the data
Feature_Selector.fit(X=X, 
                     y=y_uncuff_train, 
                     n_trials=100, 
                     sample=False, 
                     train_or_test='train', 
                     normalize=True, 
                     verbose=True, 
                     random_state=SEED)

# Plot the results of the BorutaShap feature selection
Feature_Selector.plot(X_size=10,
                       which_features='all')

In [3]:
# Exclude the 'sex' feature and concatenate 'age', 'weight', and 'height' features into a new feature set
x_uncuff_train = np.concatenate((x_uncuff_train[:,0:1], x_uncuff_train[:,2:4]),axis=-1)
x_uncuff_test = np.concatenate((x_uncuff_test[:,0:1], x_uncuff_test[:,2:4]),axis=-1)

## Traditional Formula

### Cuffed model

In [4]:
from sklearn.metrics import f1_score

# Traditional age-based formula (Duracher's formula : age / 4 + 3.5)
y_cuff_trad = np.array([math.floor((math.floor(age) / 4 + 3.5) * 2) / 2 if age >= 2 else (3.0 if age < 1 else 3.5) for age in x_cuff_test[:,0]], dtype=float)

# Get the length of the test data
total = len(x_cuff_test)

# Lists to store results from bootstrapping
y1_trads, y2_trads, f1_trads = [], [], []
runs = 10000   # Number of bootstrap runs

# Perform bootstrapping
for i in range(runs):
    # Sample random indexes with replacement
    ind = np.random.randint(total, size=total)
    
    # Create bootstrap samples for features and labels
    boot_x = x_cuff_test[ind]
    boot_y = y_cuff_test[ind]
    
    # Get the corresponding predictions from the traditional formula for bootstrapped samples
    y_trad = y_cuff_trad[ind]
    
    # Round the predicted values to the nearest 0.5 mm
    y_trad = np.round(y_trad * 2) / 2
    
    # Calculate and store accuracy metrics for the bootstrapped samples
    y1_trads.append(np.mean(y_trad == boot_y))  # Accuracy of predicting exact ETT size
    y2_trads.append(np.mean((y_trad >= boot_y - 0.5) & (y_trad <= boot_y + 0.5))) # Accuracy within tolerance of 0.5mm 
    yl_test = np.array([f'{i}' for i in boot_y])  # Convert true labels to string for f1_score calculation
    yl_pred = np.array([f'{i}' for i in y_trad])  # Convert predicted labels to string for f1_score calculation
    f1_trads.append(f1_score(yl_test, yl_pred, average='macro'))  # Calculate and store macro-averaged F1 score for each bootstrapped sample
    
# Print the results of the bootstrapping for the traditional formula for cuffed ETT
print('traditional formula for cuffed ETT (Duracher)')
print(f'acc: {np.mean(y1_trads):.3f}+-{np.std(y1_trads):.3f}, 95% CI {np.percentile(y1_trads, 5):.3f}-{np.percentile(y1_trads, 95):.3f}')
print(f'acc within 0.5mm: {np.mean(y2_trads):.3f}+-{np.std(y2_trads):.3f}, 95% CI {np.percentile(y2_trads, 5):.3f}-{np.percentile(y2_trads, 95):.3f}')
print(f'macro f1: {np.mean(f1_trads):.3f}+-{np.std(f1_trads):.3f}, 95% CI {np.percentile(f1_trads, 5):.3f}-{np.percentile(f1_trads, 95):.3f}')

traditional formula for cuffed ETT (Duracher)
acc: 0.469+-0.010, 95% CI 0.453-0.485
acc within 0.5mm: 0.966+-0.003, 95% CI 0.960-0.972
macro f1: 0.391+-0.009, 95% CI 0.377-0.405


### Uncuffed model

In [5]:
# Traditional age-based formula (Cole's formula : age / 4 + 4)
y_uncuff_trad1 = np.array([math.floor((math.floor(age) / 4 + 4) * 2) / 2 if age >= 2 else (3.5 if age < 1 else 4) for age in x_uncuff_test[:,0]], dtype=float)

# Get the length of the test data
total = len(x_uncuff_test)

# Lists to store results from bootstrapping
y1_trads, y2_trads, f1_trads = [], [], []
runs = 10000   # Number of bootstrap runs

# Perform bootstrapping
for i in range(runs):
    # Sample random indexes with replacement
    ind = np.random.randint(total, size=total)
    
    # Create bootstrap samples for features and labels
    boot_x = x_uncuff_test[ind]
    boot_y = y_uncuff_test[ind]
    
    # Get the corresponding predictions from the traditional formula for bootstrapped samples
    y_trad = y_uncuff_trad1[ind]
    
    # Round the predicted values to the nearest 0.5 mm
    y_trad = np.round(y_trad * 2) / 2
    
    # Calculate and store accuracy metrics for the bootstrapped samples
    y1_trads.append(np.mean(y_trad == boot_y))  # Accuracy of predicting exact ETT size
    y2_trads.append(np.mean((y_trad >= boot_y - 0.5) & (y_trad <= boot_y + 0.5))) # Accuracy within tolerance of 0.5mm 
    yl_test = np.array([f'{i}' for i in boot_y])  # Convert true labels to string for f1_score calculation
    yl_pred = np.array([f'{i}' for i in y_trad])  # Convert predicted labels to string for f1_score calculation
    f1_trads.append(f1_score(yl_test, yl_pred, average='macro'))  # Calculate and store macro-averaged F1 score for each bootstrapped sample
    
# Print the results of the bootstrapping for the traditional formula for uncuffed ETT
print('traditional formula for uncuffed ETT (Cole)')
print(f'acc: {np.mean(y1_trads):.3f}+-{np.std(y1_trads):.3f}, 95% CI {np.percentile(y1_trads, 5):.3f}-{np.percentile(y1_trads, 95):.3f}')
print(f'acc within 0.5mm: {np.mean(y2_trads):.3f}+-{np.std(y2_trads):.3f}, 95% CI {np.percentile(y2_trads, 5):.3f}-{np.percentile(y2_trads, 95):.3f}')
print(f'macro f1: {np.mean(f1_trads):.3f}+-{np.std(f1_trads):.3f}, 95% CI {np.percentile(f1_trads, 5):.3f}-{np.percentile(f1_trads, 95):.3f}')

traditional formula for uncuffed ETT (Cole)
acc: 0.203+-0.006, 95% CI 0.193-0.212
acc within 0.5mm: 0.781+-0.006, 95% CI 0.771-0.791
macro f1: 0.164+-0.018, 95% CI 0.140-0.196


In [6]:
# Traditional age-based formula (Penlington's formula : age / 4 + 4.5)
y_uncuff_trad2 = np.array([math.floor((math.floor(age) / 4 + 4.5) * 2) / 2 if age < 6.5 else math.floor((math.floor(age) / 3 + 3.5) * 2) / 2 for age in x_uncuff_test[:,0]], dtype=float)

# Get the length of the test data
total = len(x_uncuff_test)

# Lists to store results from bootstrapping
y1_trads, y2_trads, f1_trads = [], [], []
runs = 10000   # Number of bootstrap runs

# Perform bootstrapping
for i in range(runs):
    # Sample random indexes with replacement
    ind = np.random.randint(total, size=total)
    
    # Create bootstrap samples for features and labels
    boot_x = x_uncuff_test[ind]
    boot_y = y_uncuff_test[ind]
    
    # Get the corresponding predictions from the traditional formula for bootstrapped samples
    y_trad = y_uncuff_trad2[ind]
    
    # Round the predicted values to the nearest 0.5 mm
    y_trad = np.round(y_trad * 2) / 2
    
    # Calculate and store accuracy metrics for the bootstrapped samples
    y1_trads.append(np.mean(y_trad == boot_y))  # Accuracy of predicting exact ETT size
    y2_trads.append(np.mean((y_trad >= boot_y - 0.5) & (y_trad <= boot_y + 0.5))) # Accuracy within tolerance of 0.5mm 
    yl_test = np.array([f'{i}' for i in boot_y])  # Convert true labels to string for f1_score calculation
    yl_pred = np.array([f'{i}' for i in y_trad])  # Convert predicted labels to string for f1_score calculation
    f1_trads.append(f1_score(yl_test, yl_pred, average='macro'))  # Calculate and store macro-averaged F1 score for each bootstrapped sample
    
# Print the results of the bootstrapping for the traditional formula for uncuffed ETT
print('* traditional formula for uncuffed ETT (Penlington)')
print(f'acc: {np.mean(y1_trads):.3f}+-{np.std(y1_trads):.3f}, 95% CI {np.percentile(y1_trads, 5):.3f}-{np.percentile(y1_trads, 95):.3f}')
print(f'acc within 0.5mm: {np.mean(y2_trads):.3f}+-{np.std(y2_trads):.3f}, 95% CI {np.percentile(y2_trads, 5):.3f}-{np.percentile(y2_trads, 95):.3f}')
print(f'macro f1: {np.mean(f1_trads):.3f}+-{np.std(f1_trads):.3f}, 95% CI {np.percentile(f1_trads, 5):.3f}-{np.percentile(f1_trads, 95):.3f}')

* traditional formula for uncuffed ETT (Penlington)
acc: 0.413+-0.007, 95% CI 0.402-0.425
acc within 0.5mm: 0.826+-0.006, 95% CI 0.817-0.835
macro f1: 0.203+-0.004, 95% CI 0.196-0.211


## Linear regression model

### Cuffed model

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score

# Create a Linear Regression model for cuffed endotracheal tube data
lr_cuff = LinearRegression()

# Fit the model to the training data
lr_cuff.fit(x_cuff_train, y_cuff_train)

# Make predictions on the test data using the trained model
y_pred = lr_cuff.predict(x_cuff_test)

# Round the predicted values to the nearest 0.5 mm
y_pred = np.round(y_pred * 2) / 2

# Print the results for the Linear Regression model
print('linear regression model')
print(f'coefficient {lr_cuff.coef_}, intercept {lr_cuff.intercept_:.3f}')
print('--------------')

linear regression model
coefficient [0.21243728 0.00857616], intercept 3.558
--------------


In [8]:
# Get the length of the test data
total = len(x_cuff_test)

# Lists to store results from bootstrapping
y1_lrs, y2_lrs, f1_lrs = [], [], []
runs = 10000   # Number of bootstrap runs

# Perform bootstrapping
for i in range(runs):
    # Sample random indexes with replacement
    ind = np.random.randint(total, size=total)
    
    # Create bootstrap samples for features and labels
    boot_x = x_cuff_test[ind]
    boot_y = y_cuff_test[ind]
    
    # Calculate the prediction of linear regression model
    y_lr = lr_cuff.predict(boot_x)
    
    # Round the predicted values to the nearest 0.5 mm
    y_lr = np.round(y_lr * 2) / 2
    
    # Calculate and store accuracy metrics for the bootstrapped samples
    y1_lrs.append(np.mean(y_lr == boot_y))  # Accuracy of predicting exact ETT size
    y2_lrs.append(np.mean((y_lr >= boot_y - 0.5) & (y_lr <= boot_y + 0.5))) # Accuracy within tolerance of 0.5mm 
    yl_test = np.array([f'{i}' for i in boot_y])  # Convert true labels to string for f1_score calculation
    yl_pred = np.array([f'{i}' for i in y_lr])  # Convert predicted labels to string for f1_score calculation
    f1_lrs.append(f1_score(yl_test, yl_pred, average='macro'))  # Calculate and store macro-averaged F1 score for each bootstrapped sample
    
# Print the results of the bootstrapping for the linear regression model for cuffed ETT
print('* linear regression model for cuffed ETT')
print(f'acc: {np.mean(y1_lrs):.3f}+-{np.std(y1_lrs):.3f}, 95% CI {np.percentile(y1_lrs, 5):.3f}-{np.percentile(y1_lrs, 95):.3f}')
print(f'acc within 0.5mm: {np.mean(y2_lrs):.3f}+-{np.std(y2_lrs):.3f}, 95% CI {np.percentile(y2_lrs, 5):.3f}-{np.percentile(y2_lrs, 95):.3f}')
print(f'macro f1: {np.mean(f1_lrs):.3f}+-{np.std(f1_lrs):.3f}, 95% CI {np.percentile(f1_lrs, 5):.3f}-{np.percentile(f1_lrs, 95):.3f}')

* linear regression model for cuffed ETT
acc: 0.571+-0.010, 95% CI 0.555-0.587
acc within 0.5mm: 0.990+-0.002, 95% CI 0.987-0.993
macro f1: 0.554+-0.013, 95% CI 0.533-0.576


### Uncuffed model

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score

# Create a Linear Regression model for uncuffed endotracheal tube data
lr_uncuff = LinearRegression()

# Fit the model to the training data
lr_uncuff.fit(x_uncuff_train, y_uncuff_train)

# Make predictions on the test data using the trained model
y_pred = lr_uncuff.predict(x_uncuff_test)

# Round the predicted values to the nearest 0.5 mm
y_pred = np.round(y_pred * 2) / 2

# Print the results for the Linear Regression model
print('linear regression model')
print(f'coefficient {lr_uncuff.coef_}, intercept {lr_uncuff.intercept_:.3f}')
print('--------------')

linear regression model
coefficient [0.22728641 0.03159429], intercept 3.608
--------------


In [10]:
# Get the length of the test data
total = len(x_uncuff_test)

# Lists to store results from bootstrapping
y1_lrs, y2_lrs, f1_lrs = [], [], []
runs = 10000   # Number of bootstrap runs

# Perform bootstrapping
for i in range(runs):
    # Sample random indexes with replacement
    ind = np.random.randint(total, size=total)
    
    # Create bootstrap samples for features and labels
    boot_x = x_uncuff_test[ind]
    boot_y = y_uncuff_test[ind]
    
    # Calculate the prediction of linear regression model
    y_lr = lr_uncuff.predict(boot_x)
    
    # Round the predicted values to the nearest 0.5 mm
    y_lr = np.round(y_lr * 2) / 2
    
    # Calculate and store accuracy metrics for the bootstrapped samples
    y1_lrs.append(np.mean(y_lr == boot_y))  # Accuracy of predicting exact ETT size
    y2_lrs.append(np.mean((y_lr >= boot_y - 0.5) & (y_lr <= boot_y + 0.5))) # Accuracy within tolerance of 0.5mm 
    yl_test = np.array([f'{i}' for i in boot_y])  # Convert true labels to string for f1_score calculation
    yl_pred = np.array([f'{i}' for i in y_lr])  # Convert predicted labels to string for f1_score calculation
    f1_lrs.append(f1_score(yl_test, yl_pred, average='macro'))  # Calculate and store macro-averaged F1 score for each bootstrapped sample
    
# Print the results of the bootstrapping for the linear regression model for cuffed ETT
print('* linear regression model for uncuffed ETT')
print(f'acc: {np.mean(y1_lrs):.3f}+-{np.std(y1_lrs):.3f}, 95% CI {np.percentile(y1_lrs, 5):.3f}-{np.percentile(y1_lrs, 95):.3f}')
print(f'acc within 0.5mm: {np.mean(y2_lrs):.3f}+-{np.std(y2_lrs):.3f}, 95% CI {np.percentile(y2_lrs, 5):.3f}-{np.percentile(y2_lrs, 95):.3f}')
print(f'macro f1: {np.mean(f1_lrs):.3f}+-{np.std(f1_lrs):.3f}, 95% CI {np.percentile(f1_lrs, 5):.3f}-{np.percentile(f1_lrs, 95):.3f}')

* linear regression model for uncuffed ETT
acc: 0.409+-0.007, 95% CI 0.398-0.421
acc within 0.5mm: 0.937+-0.004, 95% CI 0.931-0.943
macro f1: 0.221+-0.005, 95% CI 0.213-0.229


## Gradient-boosted regression tree model

### Cuffed model

In [11]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, mean_squared_error, mean_absolute_error, r2_score

# age (일단위)
param_dict = {
                'learning_rate': [ 0.01, 0.03, 0.05, 0.07], #, #[0.01, 0.03, 0.05],
                'max_depth': [3, 4, 5, 7],#[3,4,5],
                'n_estimators': [25, 50, 75, 100, 300],
                'subsample': [0.5, 0.8, 1], #[0.5, 0.8, 1],
                'colsample_bytree': [0.5, 0.8, 1], #[0.8, 1],
                'gamma': [0.3, 0.5, 0.7, 0.9],
            }
nfold = 10
gs = GridSearchCV(estimator=xgb.sklearn.XGBRegressor(),
                n_jobs=-1,
                verbose=1,
                param_grid=param_dict, cv=nfold)
gs.fit(x_cuff_train, y_cuff_train)

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

gbrt_cuff = gs.best_estimator_.get_booster()

y_pred = gbrt_cuff.predict(xgb.DMatrix(x_cuff_test)).flatten()
y_pred = np.round(y_pred * 2) / 2

Fitting 10 folds for each of 2880 candidates, totalling 28800 fits

========= found hyperparameter =========
{'colsample_bytree': 1, 'gamma': 0.5, 'learning_rate': 0.07, 'max_depth': 5, 'n_estimators': 300, 'subsample': 1}
0.8739470228761033


In [12]:
# Get the length of the test data
total = len(x_cuff_test)

# Lists to store results from bootstrapping
y1_gbrts, y2_gbrts, f1_gbrts = [], [], []
runs = 10000   # Number of bootstrap runs

# Perform bootstrapping
for i in range(runs):
    # Sample random indexes with replacement
    ind = np.random.randint(total, size=total)
    
    # Create bootstrap samples for features and labels
    boot_x = x_cuff_test[ind]
    boot_y = y_cuff_test[ind]
    
    # Calculate the prediction of linear regression model
    y_gbrt = gbrt_cuff.predict(xgb.DMatrix(boot_x))
    
    # Round the predicted values to the nearest 0.5 mm
    y_gbrt = np.round(y_gbrt * 2) / 2
    
    # Calculate and store accuracy metrics for the bootstrapped samples
    y1_gbrts.append(np.mean(y_gbrt == boot_y))  # Accuracy of predicting exact ETT size
    y2_gbrts.append(np.mean((y_gbrt >= boot_y - 0.5) & (y_gbrt <= boot_y + 0.5))) # Accuracy within tolerance of 0.5mm 
    yl_test = np.array([f'{i}' for i in boot_y])  # Convert true labels to string for f1_score calculation
    yl_pred = np.array([f'{i}' for i in y_gbrt])  # Convert predicted labels to string for f1_score calculation
    f1_gbrts.append(f1_score(yl_test, yl_pred, average='macro'))  # Calculate and store macro-averaged F1 score for each bootstrapped sample
    
# Print the results of the bootstrapping for the gradient-boosted regression tree model for cuffed ETT
print('* gradient-boosted regression tree model for cuffed ETT')
print(f'acc: {np.mean(y1_gbrts):.3f}+-{np.std(y1_gbrts):.3f}, 95% CI {np.percentile(y1_gbrts, 5):.3f}-{np.percentile(y1_gbrts, 95):.3f}')
print(f'acc within 0.5mm: {np.mean(y2_gbrts):.3f}+-{np.std(y2_gbrts):.3f}, 95% CI {np.percentile(y2_gbrts, 5):.3f}-{np.percentile(y2_gbrts, 95):.3f}')
print(f'macro f1: {np.mean(f1_gbrts):.3f}+-{np.std(f1_gbrts):.3f}, 95% CI {np.percentile(f1_gbrts, 5):.3f}-{np.percentile(f1_gbrts, 95):.3f}')

* gradient-boosted regression tree model for cuffed ETT
acc: 0.701+-0.009, 95% CI 0.686-0.715
acc within 0.5mm: 0.995+-0.001, 95% CI 0.993-0.997
macro f1: 0.607+-0.008, 95% CI 0.594-0.620


### Uncuffed model

In [15]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# age (일단위)
param_dict = {
                'learning_rate': [ 0.01, 0.03, 0.05, 0.07], #, #[0.01, 0.03, 0.05],
                'max_depth': [3, 4, 5, 7],#[3,4,5],
                'n_estimators': [25, 50, 75, 100, 300],
                'subsample': [0.5, 0.8, 1], #[0.5, 0.8, 1],
                'colsample_bytree': [0.5, 0.8, 1], #[0.8, 1],
                'gamma': [0.3, 0.5, 0.7, 0.9],
            }
nfold = 10
gs = GridSearchCV(estimator=xgb.sklearn.XGBRegressor(),
                n_jobs=-1,
                verbose=1,
                param_grid=param_dict, cv=nfold)
gs.fit(x_uncuff_train, y_uncuff_train)

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

gbrt_uncuff = gs.best_estimator_.get_booster()

y_pred = gbrt_uncuff.predict(xgb.DMatrix(x_uncuff_test)).flatten()
y_pred = np.round(y_pred * 2) / 2

Fitting 10 folds for each of 2880 candidates, totalling 28800 fits

========= found hyperparameter =========
{'colsample_bytree': 1, 'gamma': 0.9, 'learning_rate': 0.03, 'max_depth': 4, 'n_estimators': 300, 'subsample': 0.8}
0.8918211310847501


In [16]:
# Get the length of the test data
total = len(x_uncuff_test)

# Lists to store results from bootstrapping
y1_gbrts, y2_gbrts, f1_gbrts = [], [], []
runs = 10000   # Number of bootstrap runs

# Perform bootstrapping
for i in range(runs):
    # Sample random indexes with replacement
    ind = np.random.randint(total, size=total)
    
    # Create bootstrap samples for features and labels
    boot_x = x_uncuff_test[ind]
    boot_y = y_uncuff_test[ind]
    
    # Calculate the prediction of linear regression model
    y_gbrt = gbrt_uncuff.predict(xgb.DMatrix(boot_x))
    
    # Round the predicted values to the nearest 0.5 mm
    y_gbrt = np.round(y_gbrt * 2) / 2
    
    # Calculate and store accuracy metrics for the bootstrapped samples
    y1_gbrts.append(np.mean(y_gbrt == boot_y))  # Accuracy of predicting exact ETT size
    y2_gbrts.append(np.mean((y_gbrt >= boot_y - 0.5) & (y_gbrt <= boot_y + 0.5))) # Accuracy within tolerance of 0.5mm 
    yl_test = np.array([f'{i}' for i in boot_y])  # Convert true labels to string for f1_score calculation
    yl_pred = np.array([f'{i}' for i in y_gbrt])  # Convert predicted labels to string for f1_score calculation
    f1_gbrts.append(f1_score(yl_test, yl_pred, average='macro'))  # Calculate and store macro-averaged F1 score for each bootstrapped sample
    
# Print the results of the bootstrapping for the gradient-boosted regression tree model for cuffed ETT
print('* gradient-boosted regression tree model for uncuffed ETT')
print(f'acc: {np.mean(y1_gbrts):.3f}+-{np.std(y1_gbrts):.3f}, 95% CI {np.percentile(y1_gbrts, 5):.3f}-{np.percentile(y1_gbrts, 95):.3f}')
print(f'acc within 0.5mm: {np.mean(y2_gbrts):.3f}+-{np.std(y2_gbrts):.3f}, 95% CI {np.percentile(y2_gbrts, 5):.3f}-{np.percentile(y2_gbrts, 95):.3f}')
print(f'macro f1: {np.mean(f1_gbrts):.3f}+-{np.std(f1_gbrts):.3f}, 95% CI {np.percentile(f1_gbrts, 5):.3f}-{np.percentile(f1_gbrts, 95):.3f}')

* gradient-boosted regression tree model for uncuffed ETT
acc: 0.581+-0.007, 95% CI 0.569-0.592
acc within 0.5mm: 0.983+-0.002, 95% CI 0.980-0.986
macro f1: 0.514+-0.031, 95% CI 0.482-0.562
